In [4]:
# rich output 출력위치 설정
# => 웹 브라우저에서 바로 봄
get_ipython().run_line_magic('matplotlib', 'inline')

# 색상 출력
# 설치 방법 => pip install termcolor
from termcolor import colored
def c(tag, data, under=3):

    val = colored(tag + ':\n', 'magenta', attrs=['bold'])
    t = type(data)
    val += colored(t, 'magenta')
    under = str(under)
    
    import numpy
    if t is float or t is numpy.float64:
        val += ('\n{:.' + under + 'f}\n').format(data)
    else:
        val += '\n{}\n'.format(data)
    return val

#### <span style="color:#184;background:#EEE">2.3.4. 나이브 베이즈 분류기</span><br>

- LogisticRegression과 LinearSVC보다 훈련 속도가 빠르지만 일반화 성능이 조금 떨어짐
- 나이브 베이즈 분류기가 효과적인 이유
    - 각 특성을 개별로 취급해 파라미터를 학습함
    - 그리고 각 특성에서 클래스별 통계를 단순하게 취합<br><br>
- scikit-learn 모델<br><br>
    
  - **BernoulliNB**
    - (이산) 이진 데이터에 적용
    - 각 클래스의 특성 중 0이 아닌 것이 몇 개인지 셈<br><br>
    
  - **MultinomialNB**
    - (이산) 카운트 데이터에 적용 (예를 들면, 문장에 나타난 단어의 횟수)
    - 클래스별로 특성의 평균을 계산<br><br>

  - **GaussianNB**
    - 연속 데이터에 적용
    - 클래스별로 각 특성의 표준편차와 평균을 저장<br><br>
    
- **<span style="color:#C22">베이즈 정리</span>**
  - $\displaystyle P(A∣B)= \frac{P(A∩B)}{P(B)}=\frac{P(A)P(B∣A)}{P(B)}$
  - $\displaystyle P(B∣A)= \frac{P(B∩A)}{P(A)}=\frac{P(B)P(A∣B)}{P(A)}$<br><br>
<img src="../images/bayes_theorem.jpg" style="margin:0;width:300px"/><br>

  
- **<span style="color:#C22">베이즈 분류</span>**<br><br>
  - 입력 데이터의 내용이 $X=(x_1, x_2, ..., x_p)$로 주어졌을 때 $Y=k$일 확률
    - $\displaystyle P(Y = k ~|~ X_1 = x_1, X_2=x_2, ... , X_p = x_p)=P(Y = k) P(X_1 = x_1, X_2=x_2, ... , X_p = x_p~|~Y = k)$<br><br>
  - 입력 데이터의 각 특성값의 조건부 분포가 서로 독립이라는 단순 베이즈 가정을 한다면
    - $\displaystyle P(Y = k ~|~ X_1 = x_1, X_2=x_2, ... , X_p = x_p)=P(Y = k) \prod_{j=1}^{p}P(X_j = x_j~|~Y = k)$<br><br>
  - 따라서, 입력 데이터의 내용이 $X=(x_1, x_2, ..., x_p)$로 주어졌을 때 $Y$ 예측 방법
    - $\displaystyle argmax_{k\in Y} \big{(}P(Y = k) \prod_{j=1}^{p}P(X_j = x_j~|~Y = k)\big{)}$

In [9]:
# BernoulliNB 분류기만 해보도록 함
import numpy as np
from sklearn.naive_bayes import BernoulliNB

X_train = np.array([[0, 1, 0, 1],
              [1, 0, 1, 1],
              [0, 0, 0, 1],
              [1, 0, 1, 0]])

y_train = np.array([0, 1, 0, 1])

# 특성 카운트를 직접 해보기
counts = {}
for label in np.unique(y_train):
    # 특성이 있으면 다 1이기 때문에 그냥 더하면 됨
    counts[label] = X_train[y_train == label].sum(axis=0)

# 모델 생성 및 학습
clf = BernoulliNB()
clf.fit(X_train, y_train)

y_test = [[0, 0, 0, 0],
          [1, 1, 1, 1]]

print(c('feature counts', counts))
print(c('prediction', clf.predict(y_test)))

feature counts:
<class 'dict'>
{0: array([0, 1, 0, 2]), 1: array([2, 0, 2, 1])}

prediction:
<class 'numpy.ndarray'>
[0 1]



- **<span style="color:#C22">장단점과 매개변수</span>**<br><br>
    
    - **설명**
        1. BernoulliNB와 MultinomialNB는 alpah 값을 매개변수로 설정 가능
        2. alpha 값만큼 모든 특성에 양의 값을 가진 가상의 데이터 포인트를 추가
        3. 이는 통계 데이터를 완만하게 해줌, alpha가 클수록 모델의 복잡도는 저하
        4. GaussianNB는 주로 고차원 데이터 셋, 나머지는 텍스트와 같이 희소 데이터 셋
        5. MultinomialNB는 매우 큰 문서들에게 효과적<br><br>
    
    - **강점**
        1. 훈련과 예측 속도가 빠름
        2. 희소 고차원 데이터 셋에서 잘 작동
        3. 매개변수에 민감하지 않음
        4. 다른 선형 모델로 하면 너무 오래 걸릴 때 종종 시도함